# Overview

There are two evolutionary algorithms built into TPOT2, which corresponds to two different estimator classes.

1. The `tpot2.TPOTEstimator` uses a standard evolutionary algorithm that evaluates exactly population_size individuals each generation. This is similar to the algorithm in TPOT1. The next generation does not start until the previous is completely finished evaluating. This leads to underutilized CPU time as the cores are waiting for the last individuals to finish training, but may preserve diversity in the population. 

2. The `tpot2.TPOTEstimatorSteadyState` differs in that it will generate and evaluate the next individual as soon as an individual finishes evaluation. The number of individuals being evaluated is determined by the n_jobs parameter. There is no longer a concept of generations. The population_size parameter now refers to the size of the list of evaluated parents. When an individual is evaluated, the selection method updates the list of parents. This allows more efficient utilization when using multiple cores.


Additionally, two other simplified estimators are provided. These have a simplified set of hyperparameters with default values set for classification and regression problems. Currently, both of these use the standard evolutionary algorithm in the `tpot2.TPOTEstimator` class.

1. `tpot2.TPOTClassifier` for classification tasks
2. `tpot2.TPOTRegressor` for regression tasks

#### Scorers, Objective Functions, and multi objective optimization.

There are two ways of passing objectives into TPOT2. 

1. `scorers`: Scorers are functions that have the signature (estimator, X, y). These can be produced with the [sklearn.metrics.make_scorer](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) function. This function is used to evaluate the test folds during cross validation. These are passed into TPOT2 via the scorers parameter. This can take in the scorer itself or the string corresponding to a scoring function ([as listed here](https://scikit-learn.org/stable/modules/model_evaluation.html)). TPOT2 also supports passing in a list of several scorers for multiobjective optimization. 

2. `other_objective_functions` : Other objective functions in TPOT2 have the signature (estimator) and returns a float or list of floats. These get passed an unfitted estimator (in the case of TPOT2, a `tpot2.GraphPipeline`). 


Each scorer and objective function must be accompanied by a list of weights corresponding to the list of objectives. By default, TPOT2 maximizes objective functions (this can be changed by `bigger_is_better=False`). Positive weights means that TPOT2 will seek to maximize that objective, and negative weights correspond to minimization.

Here is an example of using two scorers

    scorers=['roc_auc_ovr',tpot2.objectives.complexity_scorer],
    scorers_weights=[1,-1],


Here is an example with a scorer and a secondary objective function

    scorers=['roc_auc_ovr'],
    scorers_weights=[1],
    other_objective_functions=[tpot2.objectives.number_of_leaves_objective],
    other_objective_functions_weights=[-1],

In [1]:
import tpot2
import sklearn
import sklearn.datasets

scorer = sklearn.metrics.get_scorer('roc_auc_ovo')
X, y = sklearn.datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.75, test_size=0.25)


est = tpot2.TPOTClassifier(n_jobs=4, max_time_seconds=60, verbose=2)
est.fit(X_train, y_train)


print(scorer(est, X_test, y_test))

Generation: : 0it [00:00, ?it/s]/home/ribeirop/common/Projects/TPOT_Dev/tpot2/tpot2/population.py:204: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.9987673084964015, 0.9847336570645737, 0.9905041856662168, 0.9993763441765191, 0.9907964721472181, 0.9972029119373254, 0.998357673885215, 0.9990613894532178, 0.9694329382784964, 0.997532770251522, 0.9975246891516141, 0.9904661635101079, 0.5, 'INVALID', 0.9948797140575193, 0.5, 0.9849666992163886, 0.9824262822238007, 0.9985479308254266, 0.9972915114620106, 0.9694329382784964, 'INVALID', 0.9324417946451, 0.9901685392720255, 0.9978183481741485, 0.9973644125394717, 0.9905750509316356, 0.9819493383116706, 0.9699621501061083, 0.6072655018077077, 0.9694329382784964, 0.9838996235635504, 0.982122385127114, 0.9901266523287818, 0.9301526525124777, 0.9720743554304064, 0.994576960473181, 0.5, 0.5, 0.9948330115435265, 0.9990358447113457, 0.9945434259359371, 0.9375978779

0.9999588595957566


In [1]:
import tpot2
import sklearn
import sklearn.metrics
import sklearn.datasets

scorer = sklearn.metrics.get_scorer('neg_mean_squared_error')
X, y = sklearn.datasets.load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.75, test_size=0.25)

est = tpot2.tpot_estimator.templates.TPOTRegressor(n_jobs=4, max_time_seconds=30, verbose=2)
est.fit(X_train, y_train)

print(scorer(est, X_test, y_test))

Generation: : 1it [00:03,  3.46s/it]/home/ribeirop/common/Projects/TPOT_Dev/tpot2/tpot2/population.py:381: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'ind_mutate' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.evaluated_individuals.at[new_child.unique_id(),"Variation_Function"] = var_op
/home/ribeirop/common/Projects/TPOT_Dev/tpot2/tpot2/population.py:204: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['INVALID', -6039.425686838566, -3697.10004861336, -3609.502376363637, -5283.160282257095, -3081.9675902170966, -3159.936813075531, -6.206059150347736e+26, -2983.0962978018993, -3310.997549085264, -3255.911452949286, -3437.783411085481, -6124.753602783256, -2983.0962978018993, 'INVALID', -2951.7123134502217, -3172.715957996382, -3203.3174204236266, -3172.204242312273, -3132.3

-3453.3557493847698


### Best Practices

When running tpot from an .py script, it is important to protect code with `if __name__=="__main__":`

In [ ]:
#my_analysis.py

from dask.distributed import Client, LocalCluster
import tpot2
import sklearn
import sklearn.datasets
import numpy as np

if __name__=="__main__":
    scorer = sklearn.metrics.get_scorer('roc_auc_ovo')
    X, y = sklearn.datasets.load_digits(return_X_y=True)
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.75, test_size=0.25)


    est = tpot2.TPOTClassifier(n_jobs=4, max_time_seconds=60, verbose=2)
    est.fit(X_train, y_train)


    print(scorer(est, X_test, y_test))

### Common parameters

        scorers : (list, scorer)
            A scorer or list of scorers to be used in the cross-validation process. 
            see https://scikit-learn.org/stable/modules/model_evaluation.html
        
        scorers_weights : list
            A list of weights to be applied to the scorers during the optimization process.
        
        classification : bool
            If True, the problem is treated as a classification problem. If False, the problem is treated as a regression problem.
            Used to determine the CV strategy.
        
        cv : int, cross-validator
            - (int): Number of folds to use in the cross-validation process. By uses the sklearn.model_selection.KFold cross-validator for regression and StratifiedKFold for classification. In both cases, shuffled is set to True.
            - (sklearn.model_selection.BaseCrossValidator): A cross-validator to use in the cross-validation process.
                - max_depth (int): The maximum depth from any node to the root of the pipelines to be generated.
        
        other_objective_functions : list, default=[tpot2.objectives.estimator_objective_functions.average_path_length_objective]
            A list of other objective functions to apply to the pipeline.
        
        other_objective_functions_weights : list, default=[-1]
            A list of weights to be applied to the other objective functions.
        
        objective_function_names : list, default=None
            A list of names to be applied to the objective functions. If None, will use the names of the objective functions.
        
        bigger_is_better : bool, default=True
            If True, the objective function is maximized. If False, the objective function is minimized. Use negative weights to reverse the direction.
        
        generations : int, default=50
            Number of generations to run
            
        max_time_seconds : float, default=float("inf")
            Maximum time to run the optimization. If none or inf, will run until the end of the generations.
        
        max_eval_time_seconds : float, default=60*5
            Maximum time to evaluate a single individual. If none or inf, there will be no time limit per evaluation.

        n_jobs : int, default=1
            Number of processes to run in parallel.
        
        memory_limit : str, default="4GB"
            Memory limit for each job. See Dask [LocalCluster documentation](https://distributed.dask.org/en/stable/api.html#distributed.Client) for more information.

            
        verbose : int, default=1 
            How much information to print during the optimization process. Higher values include the information from lower values.
            0. nothing
            1. progress bar
            
            3. best individual
            4. warnings
            >=5. full warnings trace
            6. evaluations progress bar. (Temporary: This used to be 2. Currently, using evaluation progress bar may prevent some instances were we terminate a generation early due to it reaching max_time_seconds in the middle of a generation OR a pipeline failed to be terminated normally and we need to manually terminate it.)
        


# TPOTEstimator and TPOTEstimatorSteadyState

TPOTEstimator and TPOTEstimatorSteadyState expose more parameters for customizing search spaces and evolutionary algorithms. The next tutorial will cover customizing search spaces in more detail.

The TPOTClassifier and TPOTRegressor set default parameters for the TPOTEstimator for Classification and Regression.
In the future, a metalearner will be used to predict the best values for a given dataset.

### tpot2.TPOTEstimatorSteadyState

In [ ]:
import tpot2
import sklearn
import sklearn.datasets


graph_search_space = tpot2.search_spaces.pipelines.GraphPipeline(
    root_search_space= tpot2.config.get_search_space(["KNeighborsClassifier", "LogisticRegression", "DecisionTreeClassifier"]),
    leaf_search_space = tpot2.config.get_search_space("selectors"), 
    inner_search_space = tpot2.config.get_search_space(["transformers"]),
    max_size = 10,
)

est = tpot2.TPOTEstimatorSteadyState( 
                            search_space = graph_search_space,
                            scorers=['roc_auc_ovr'], #scorers can be a list of strings or a list of scorers. These get evaluated during cross validation. 
                            scorers_weights=[1],

                            classification=True,

                            max_eval_time_seconds=15,
                            max_time_seconds=30,
                            verbose=2)


scorer = sklearn.metrics.get_scorer('roc_auc_ovo')
X, y = sklearn.datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.75, test_size=0.25)
est.fit(X_train, y_train)
print(scorer(est, X_test, y_test))


In [ ]:
fitted_pipeline = est.fitted_pipeline_ # access best pipeline directly
fitted_pipeline.plot()

In [ ]:
#view the summary of all evaluated individuals as a pandas dataframe
est.evaluated_individuals

In [ ]:
import tpot2
import sklearn
import sklearn.datasets

est = tpot2.TPOTEstimatorSteadyState( 
                            search_space = graph_search_space,
                            scorers=['roc_auc_ovr',tpot2.objectives.complexity_scorer],
                            scorers_weights=[1,-1],

                            classification=True,

                            max_eval_time_seconds=15,
                            max_time_seconds=30,
                            verbose=2)


scorer = sklearn.metrics.get_scorer('roc_auc_ovo')
X, y = sklearn.datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.75, test_size=0.25)
est.fit(X_train, y_train)
print(scorer(est, X_test, y_test))


In [ ]:
fitted_pipeline = est.fitted_pipeline_ # access best pipeline directly
fitted_pipeline.plot() #plot the best pipeline

view the results of all evaluated individuals as a pandas dataframe

In [ ]:
est.evaluated_individuals

view pareto front as a pandas dataframe

In [ ]:
est.pareto_front

In [ ]:
pareto_front = est.pareto_front

#plot the pareto front of number_of_leaves_objective vs roc_auc_score

import matplotlib.pyplot as plt
plt.scatter(pareto_front['complexity_scorer'], pareto_front['roc_auc_score'])
plt.xlabel('complexity_scorer')
plt.ylabel('roc_auc_score')
plt.show()

### tpot2.TPOTEstimator

In [ ]:
import tpot2
import sklearn
import sklearn.datasets

est = tpot2.TPOTEstimator(  
                            search_space = graph_search_space,
                            population_size=30,
                            generations=5,
                            scorers=['roc_auc_ovr'], #scorers can be a list of strings or a list of scorers. These get evaluated during cross validation. 
                            scorers_weights=[1],
                            classification=True,
                            n_jobs=1, 
                            early_stop=5, #how many generations with no improvement to stop after
                            
                            #List of other objective functions. All objective functions take in an untrained GraphPipeline and return a score or a list of scores
                            other_objective_functions= [ ],
                            
                            #List of weights for the other objective functions. Must be the same length as other_objective_functions. By default, bigger is better is set to True. 
                            other_objective_functions_weights=[],
                            verbose=2)

scorer = sklearn.metrics.get_scorer('roc_auc_ovo')
X, y = sklearn.datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.75, test_size=0.25)
est.fit(X_train, y_train)
print(scorer(est, X_test, y_test))